In [5]:
import pandas as pd
df = pd.read_csv(r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\Paygab by industry.csv')

columns_to_drop = [
    'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME',
    'Time frequency', 'Unit of measure',
    'nace_r2',
    'Geopolitical entity (reporting)', 'Time', 'Observation value',
    'Observation status (Flag) V2 structure', 'Confidentiality status (flag)'
]

df_cleaned = df.drop(columns=columns_to_drop)

df = df.rename(columns={
    'Statistical classification of economic activities in the European Community (NACE Rev. 2)': 'industry'
})
industry_gap = df.groupby('industry')['OBS_VALUE'].mean().sort_values(ascending=False)

industry_gap.head(10)


industry
Financial and insurance activities                                      28.075957
Wholesale and retail trade; repair of motor vehicles and motorcycles    20.387185
Information and communication                                           19.160510
Manufacturing                                                           19.135532
Professional, scientific and technical activities                       18.666958
Human health and social work activities                                 18.530189
Arts, entertainment and recreation                                      17.417568
Business economy                                                        16.977059
Other service activities                                                16.729688
Real estate activities                                                  14.149776
Name: OBS_VALUE, dtype: float64

In [13]:
import pdfplumber
import pandas as pd
import re

pdf_path = r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\sic2007.pdf'

sic_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        lines = text.split('\n')
        for line in lines:
            # Example pattern: "85310 General secondary education"
            parts = line.strip().split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                sic_data.append(parts)

# Convert to DataFrame
sic_df = pd.DataFrame(sic_data, columns=['sic_code', 'industry_name'])

sic_df.head()

,sic_code,industry_name
0,175,"Fifth Avenue, New York, NY 10010."
1,10,9 8 7 6 5 4 3 2 1
2,18,17 16 15 14 13 12 11 10 09
3,2007,2003 2007 2003
4,01160,01110 Abaca and other vegetable textile fibre ...


In [5]:
import pdfplumber
import pandas as pd
import re

# Load PDF and CSV
pdf_path = r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\sic2007.pdf'
df1 = pd.read_csv(r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\UKpg.csv')

# Extract SIC data from PDF
sic_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        lines = text.split('\n')
        for line in lines:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                sic_data.append(parts)

# Convert to DataFrame
sic_data_df = pd.DataFrame(sic_data, columns=['sic_code', 'industry_name'])

# Clean SIC mapping (deduplicate and strip spaces)
sic_cleaned_df = sic_data_df.drop_duplicates(subset='sic_code', keep='first').copy()
sic_cleaned_df.loc[:, 'industry_name'] = sic_cleaned_df['industry_name'].str.strip()

# Prepare merge keys
df1['sic_codes'] = df1['sic_codes'].astype(str)
sic_cleaned_df['sic_code'] = sic_cleaned_df['sic_code'].astype(str)

# Merge cleaned SIC descriptions into pay gap data
df1_merged = df1.merge(sic_cleaned_df, left_on='sic_codes', right_on='sic_code', how='left')

# Preview
df1_merged[['employer_name', 'sic_codes', 'industry_name']].info()

df1.info()
                                       

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48711 entries, 0 to 48710
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   employer_name  48711 non-null  object
 1   sic_codes      48711 non-null  object
 2   industry_name  29010 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48711 entries, 0 to 48710
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   employer_name                 48711 non-null  object 
 1   employer_id                   48711 non-null  int64  
 2   address                       48681 non-null  object 
 3   post_code                     48547 non-null  object 
 4   company_number                42311 non-null  object 
 5   sic_codes                     48711 non-null  object 
 6   diff_mean_hourly_percent      48711 non-null  float